In [13]:
import paramiko
from getpass import getpass
import os, os.path

# Check to see if ssh_results file exists
if os.path.exists('doomp.txt'):
    # Ask user if they wish to delete the file (not overwrite.)
    user_input = input("There is a file under that name already. Want to delete it, sire? (y/n) ")

    # If yes, deletes the file.
    if user_input == "y":
        os.remove('doomp.txt')

# Create password prompt
thePass = getpass(prompt="Please enter your SSH password.")

# Host information
host = "45.62.254.113"
port = 22
username = "root"
password = thePass

try :
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host,port,username,password)
except paramiko.AuthenticationException:
    print ("Authentication failed")

In [14]:
# start SFTP session
sftp = ssh.open_sftp()

# The local file in our directory
localFile = "fs.py"
# The remote path to upload the file to
remotePath = "/home/john/fs.py"

# push the file to the linux server in the correct path
sftp.put(localFile, remotePath)

# Run the file that was pushed
command = "python3 fs.py -d /usr/bin"

# Get the output from the command
stdin, stdout, stderr = ssh.exec_command(command)

# Results from stdout
lines = stdout.readlines()

# Convert the list to a string
output = ''.join(lines)
# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'
# Footer
sepFooter = '' + '### END ' + command  + ' ###\n\n'
# Concatenates the header, command, footer
cmd_output = sepHeader + output + sepFooter

# Save the cmd_output to a file
with open('doomp.txt', 'a') as f:
    f.write(cmd_output)


In [15]:
# Close SSH
ssh.close()